# Summary:

## Goal:
Given the data of music albums from [CSM](https://www.commonsensemedia.org/music-reviews), the purpose of this notebook is to find the corresponding album identifiers in Spotify.    

## Tools:
-  Spotify API and its python interface Spotipy.
-  Python regular expression to clean string and match

## Outcomes:
I found 1748 album ids out of 2206 albums.

# Import modules and credentials

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import spotipy
import os
import sys
import re
import time 
%matplotlib inline

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

from dotenv import load_dotenv
%load_ext dotenv
%dotenv


client_id = os.environ['SPOTIFY_CLIENT_ID']
client_secret = os.environ['SPOTIFY_CLIENT_SECRET']
spotify = spotipy.Spotify(
    client_credentials_manager=SpotifyClientCredentials(client_id = client_id, client_secret=client_secret))

# Load  album age ratings

In [2]:
album_ratings = pd.read_csv("../data/album_age_ratings.csv")
print ("Number of albums: ", album_ratings.shape[0])
display(album_ratings.sample(3))

Number of albums:  2206


,Age,Album_Name,Artist,Year,Description,Age Group
632,8,A Little Bit Longer,Jonas Brothers,2008,"OMG! More of, like, the same sweet punk pop!",Middle Childhood 1
339,4,My Trampoline,Peter Himmelman,2009,"Clever lyrics and bright, stylish music in a ...",Pre-Schooler
1,2,A Child's Celebration of Rock 'n' Roll,Various Artists,1996,Songs to get toddlers and teens energized.,Toodler


# Search albums using SpotifyAPI

In [3]:
# Example: write search query

album_name = "Can a Jumbo Jet Sing the Alphabet?"
artist_name = "Hap Palmer"

results = spotify.search(q="album:" + album_name + " artist:" + artist_name, type='album,track')

print (results['albums']['items'][0]['name'])
print (results['albums']['items'][0]['id'])
print (results['albums']['items'][0]['artists'])


Can a Jumbo Jet Sing the Alphabet? - Songs For Learning Through Music and Movement
5HNNn1QUjSqI3UaBs2uc5f
[{'external_urls': {'spotify': 'https://open.spotify.com/artist/69mxXDPodcz1qr9p68AqsJ'}, 'href': 'https://api.spotify.com/v1/artists/69mxXDPodcz1qr9p68AqsJ', 'id': '69mxXDPodcz1qr9p68AqsJ', 'name': 'Hap Palmer', 'type': 'artist', 'uri': 'spotify:artist:69mxXDPodcz1qr9p68AqsJ'}]


In [17]:
# Example: Search for a track


track_name = "I Found You" 
artist_name = "The Wanted"


results = spotify.search(q="track:" + track_name + " artist:" + artist_name, type='track')

print (results['tracks']['items'][0]['album']['id'])
print (results['tracks']['items'][0]['album']['name'])
print (results['tracks']['items'][0]['name'])
print (results['tracks']['items'][0]['id'])


4kOQZmvSBepp9AUhc65Tgf
Word Of Mouth (Deluxe)
I Found You
0U7s0q2Q81a3n4bvPi6vSb


#  Define string-matchinig methods

In [22]:


def isMatch(str1, str2, clean_str = None):
    
    """
    Decide whether str1 and str2 matched in a loose sense.
    """
    if (str1.lower() in str2.lower()) or (str2.lower() in str1.lower()):
        return True
    
    if clean_str:
        str1, str2 = clean_str(str1), clean_str(str2)
        return (str1 in str2) or (str2 in str1)
        


def manualMatch(title1, title2, artist1, artist2):
    """
    Let users decide whether (title1, artist1) and (title2, artist2) is a match.
    """

    

    display(pd.DataFrame([[title1, artist1],[title2, artist2]], index = ['query input','output']))
    
    match = input("Is it match? (y/n)")
    return (match.lower()=='y')

#  Search Albums: Found 953 albums

In [4]:
def search_album(album_name, artist_name, loose_query = False, manual_match = False, clean_str = None):
    
    if clean_str:
        album_name, artist_name = clean_str(album_name), clean_str(artist_name)
    
    if loose_query:
        results = spotify.search(q=album_name, type='album')  # slower
    else:
        results = spotify.search(q="album:" + album_name + " artist:" + artist_name, type='album') # faster

    for item in results['albums']['items']:
        album_name_spotify = item['name']
        album_id = item['id']
        artists = [artist['name'] for artist in item['artists']]
        
        if manual_match:
            for artist in artists:
                search_terms = [album_name, album_name_spotify, artist_name, artist]
                if manualMatch(*search_terms):
                    return [album_id, album_name_spotify, artists]
        
        elif isMatch(album_name, album_name_spotify):
            for artist in artists: 
                if isMatch(artist_name, artist, clean_str):
                    return [album_id, album_name_spotify, artists]
                 

In [5]:
results = []
t1 = time.time()
    
for (i, row) in album_ratings.iterrows():
    
    if i%300 ==0 :
        time_elapsed = (time.time()-t1)/60
        print (f"Processing {i}th row.  Time elapsed: {time_elapsed:.2f} minutes. Found {len(results)} albums.")
        
    album_name, artist_name = row.Album_Name, row.Artist
   
    if not 'cd single' in album_name.lower():
        try:
            result = search_album(album_name, artist_name)

            if result: 
                results.append((album_name, artist_name, *result))
        except:
            print("(i, album_name, artist_name)=", (i,album_name, artist_name))
            print("Unexpected error:", sys.exc_info()[0])
            break
    
    
print (f"Found {len(results)} Albums.")

albums_id_1 = pd.DataFrame(results,columns=['Album_Name','Artist','Album_ID','Album_Name_Spotify','Artists_Spotify'])

Processing 0th row.  Time elapsed: 0.00 minutes. Found 0 albums.
Processing 300th row.  Time elapsed: 0.58 minutes. Found 157 albums.
Processing 600th row.  Time elapsed: 1.03 minutes. Found 252 albums.
Processing 900th row.  Time elapsed: 1.37 minutes. Found 358 albums.
Processing 1200th row.  Time elapsed: 1.67 minutes. Found 452 albums.
Processing 1500th row.  Time elapsed: 2.12 minutes. Found 607 albums.
Processing 1800th row.  Time elapsed: 2.53 minutes. Found 758 albums.
Processing 2100th row.  Time elapsed: 2.88 minutes. Found 875 albums.
Found 953 Albums.


In [5]:
#albums_id_1.to_csv("../data/albums_id_1.csv", index=False)

albums_id_1 = pd.read_csv("../data/albums_id_1.csv")
print ("Number of albums found: ", albums_id_1.shape[0])

# Albums name 
mask = albums_id_1['Album_Name']!=albums_id_1['Album_Name_Spotify']
display(albums_id_1[mask].sample(3))

Number of albums found:  953


,Album_Name,Artist,Album_ID,Album_Name_Spotify,Artists_Spotify
639,Best of Hilary Duff,Hilary Duff,3w94bw5d71KiF3NOkxSwPi,Best Of,['Hilary Duff']
426,Life for Rent,Dido,6PEApt1cBh2EtWeXMKZJzo,Life For Rent,['Dido']
920,Curtain Call,Eminem,5qENHeCSlwWpEzb25peRmQ,Curtain Call: The Hits (Deluxe Edition),['Eminem']


# Search Albums with clean string, loose search: Found  125 albums

In [23]:
def cleanStr(s):
   
    """
    Clean input string by keeping only alphabet, numerics, and white space. Then lowercasing.
    """
    
    # https://stackoverflow.com/a/70310018
    
    pattern = "\(.*\)"
    pattern += "|Original Motion Picture Soundtrack|Original Soundtrack|Soundtrack"
    pattern += "|Soundtrack Album|Movie Soundtrack|Music from the Motion Picture|Original Motion Picture"
    s = re.sub(pattern, "", s, flags = re.IGNORECASE).lower().strip()
    
    #del_chars = ''.join(c for c in map(chr, range(256)) if not (c.isalnum()|c.isspace()))
    #del_map = str.maketrans('', '', del_chars)
    #scrunched = s.translate(del_map).lower().strip()
                        
    #return scrunched
    return s

In [136]:
results = []
t1 = time.time()
found = albums_id_1[['Album_Name','Artist']].values.tolist() 

for (i, row) in album_ratings.iterrows():
    
    if i%200 ==0 :
        time_elapsed = (time.time()-t1)/60
        print (f"Processing {i}th row.  Time elapsed: {time_elapsed:.2f} minutes. Found {len(results)} albums.")
        
    album_name, artist_name = row.Title, row.Artist
    
    if [album_name, artist_name] in found:  # ignore albums already found
        continue
    
    
    if not 'cd single' in album_name.lower():
        result = search_album(album_name, artist_name, loose_query = True, clean_str = cleanStr)
        if result: 
            results.append((album_name, artist_name, *result))
      
    
print (f"Found {len(results)} Albums.")

albums_id_2 = pd.DataFrame(results,columns=['Album_Name','Artist','Album_ID','Album_Name_Spotify','Artists_Spotify'])

Processing 0th row.  Time elapsed: 0.00 minutes. Found 0 albums.
Processing 200th row.  Time elapsed: 0.21 minutes. Found 6 albums.
Processing 400th row.  Time elapsed: 0.39 minutes. Found 18 albums.
Processing 600th row.  Time elapsed: 0.57 minutes. Found 52 albums.
Processing 800th row.  Time elapsed: 0.69 minutes. Found 72 albums.
Processing 1000th row.  Time elapsed: 0.78 minutes. Found 79 albums.
Processing 1200th row.  Time elapsed: 0.85 minutes. Found 85 albums.
Processing 1400th row.  Time elapsed: 0.93 minutes. Found 96 albums.
Processing 1600th row.  Time elapsed: 0.98 minutes. Found 106 albums.
Processing 1800th row.  Time elapsed: 1.06 minutes. Found 113 albums.
Processing 2000th row.  Time elapsed: 1.13 minutes. Found 120 albums.
Processing 2200th row.  Time elapsed: 1.19 minutes. Found 125 albums.
Found 125 Albums.


In [6]:
#albums_id_2.to_csv("../data/albums_id_2.csv", index=False)

albums_id_2 = pd.read_csv("../data/albums_id_2.csv")
print ("Number of albums found: ", albums_id_2.shape[0])

# Albums name 
mask = albums_id_2['Album_Name']!=albums_id_2['Album_Name_Spotify']
display(albums_id_2[mask].sample(3))

Number of albums found:  125


,Album_Name,Artist,Album_ID,Album_Name_Spotify,Artists_Spotify
12,The Princess and the Frog Soundtrack,Various Artists,0CcL28OkH89kjgKpNZC8sW,The Princess and the Frog (Original Motion Pic...,['Various Artists']
113,Taking Woodstock Movie Soundtrack,Various Artists,6oN7rDjXg03uGblQzuk52p,Taking Woodstock [Original Motion Picture Soun...,['Various Artists']
28,A Cinderella Story Soundtrack,Various Artists,5fDujJMK88H9vZYRjVb2Bz,A Cinderella Story: Once Upon A Song (Original...,['Various Artists']


# Search Singles:  Found 567 singles

In [24]:
def search_single(album_name, artist_name, loose_query = False, manual_match = False):
    
    track_name = album_name.lower().replace("(cd single)","").replace("\"","").strip()
    
    if loose_query:
        results =  spotify.search(q="track:" + track_name, type='track')
    else:
        results =  spotify.search(q="track:" + track_name + " artist:" + artist_name, type='track')
        
    for item in results['tracks']['items']:
        track_name_spotify = item['name']
        track_id = item['id']
        album_name = item['album']['name']
        album_id = item['album']['id']
        artists = [artist['name'] for artist in item['artists']]
                
        if manual_match:
            for artist in artists:
                search_terms = [album_name, track_name_spotify, artist_name, artist]         
                if manualMatch(*search_terms):
                    return [album_id, album_name, artists, track_id, track_name_spotify]
        
        elif isMatch(track_name, track_name_spotify):
            for artist in artists: 
                if isMatch(artist, artist_name):
                    return [album_id, album_name, artists, track_id, track_name_spotify]


In [25]:
results = []
t1 = time.time()

found = albums_id_1[['Album_Name','Artist']].values.tolist() + albums_id_2[['Album_Name','Artist']].values.tolist()

for (i, row) in album_ratings.iterrows():
    
    if i%400 ==0 :
        time_elapsed = (time.time()-t1)/60
        print (f"Processing {i}th row.  Time elapsed: {time_elapsed:.2f} minutes. Found {len(results)} singles.")
        
    album_name, artist_name = row.Album_Name, row.Artist
    
    if ([album_name, artist_name] not in found) and ('cd single' in album_name.lower()):  
        result = search_single(album_name, artist_name)
        if result: 
            results.append((album_name, artist_name, *result))
        
        else:
            result = search_single(album_name, artist_name, loose_query = True)
            if result: 
                results.append((album_name, artist_name, *result))

    
print (f"Found {len(results)} Singles.")



Processing 0th row.  Time elapsed: 0.00 minutes. Found 0 singles.
Processing 400th row.  Time elapsed: 0.03 minutes. Found 7 singles.
Processing 800th row.  Time elapsed: 0.43 minutes. Found 129 singles.
Processing 1200th row.  Time elapsed: 0.86 minutes. Found 295 singles.
Processing 1600th row.  Time elapsed: 1.20 minutes. Found 398 singles.
Processing 2000th row.  Time elapsed: 1.62 minutes. Found 526 singles.
Found 567 Singles.


In [26]:
singles_id = pd.DataFrame(results,columns=['Album_Name','Artist','Album_ID','Album_Name_Spotify','Artists_Spotify','Track_ID',
                                             'Track_Name_Spotify'])
singles_id.sample(3)

,Album_Name,Artist,Album_ID,Album_Name_Spotify,Artists_Spotify,Track_ID,Track_Name_Spotify
124,"""Skyscraper"" (CD Single)",Demi Lovato,1ypH0eU9RcE6wngSGSqmeY,Unbroken,[Demi Lovato],4B3RmT3cGvh8By3WY9pbIx,Skyscraper
474,"""Scream"" (CD Single)",Usher,7eyuQZCLG5tdjWpKZbDE8g,Looking 4 Myself (Expanded Edition),[Usher],12eMQUn2GzfsfsQLH53M19,Scream
409,"""How to Love"" (CD Single)",Lil Wayne,1MGEMBqM3v8OUM2glv7MSx,Tha Carter IV,[Lil Wayne],36IZGm2IuECLTxMKa9qajO,How To Love


In [31]:
singles_id.to_csv("../data/single_ids.csv", index=False)

singles_id= pd.read_csv("../data/single_ids.csv")
print ("Number of singles found: ", singles_id.shape[0])

display(singles_id.sample(3))

Number of singles found:  567


,Album_Name,Artist,Album_ID,Album_Name_Spotify,Artists_Spotify,Track_ID,Track_Name_Spotify
338,"""Holocene"" (CD Single)",Bon Iver,2LpfNj3vB5rOXfaawLcOBg,Bon Iver,['Bon Iver'],35KiiILklye1JRRctaLUb4,Holocene
77,"""Like My Mother Does"" (CD Single)",Lauren Alaina,2KdVCn9Q0Q7UQd2L8SpcPX,Wildflower,['Lauren Alaina'],7t8oQycrEftd8NuWdCcfQb,Like My Mother Does
151,"""Kissin U"" (CD single)",Miranda Cosgrove,3Y5W2onbNSiRWSwROthGnH,Sparks Fly (Deluxe Version),['Miranda Cosgrove'],3pt4BDnaMo2vznoVSv7b2y,Kissin U


# Semi-Manual Search:  Display query result and decide match or not

In [11]:
results = []
t1 = time.time()

found = albums_id_1[['Album_Name','Artist']].values.tolist() + albums_id_2[['Album_Name','Artist']].values.tolist()
found += singles_id_1[['Album_Name', 'Artist']].values.tolist()

print (f"Found {len(found)} albums; {len(album_ratings)-len(found)} albums not found yet.")

Found 1645 albums; 561 albums not found yet.


In [ ]:
for (i, row) in album_ratings.iterrows():
    
    if i%400 ==0 :
        time_elapsed = (time.time()-t1)/60
        print (f"Processing {i}th row.  Time elapsed: {time_elapsed:.2f} minutes. Found {len(results)} singles.")
        
    album_name, artist_name = row.Album_Name, row.Artist
    
    if ([album_name, artist_name] not in found) and ('cd single' not in album_name.lower()):  
        result = search_album(album_name, artist_name, loose_query = True, manual_match = True)
        if result: 
            results.append((album_name, artist_name, *result))
        
    
print (f"Semi-manually Found {len(results)} albums.")

albums_id_3 = pd.DataFrame(results,columns=['Album_Name','Artist','Album_ID','Album_Name_Spotify','Artists_Spotify'])

In [167]:
albums_id_3 = pd.DataFrame(results,columns=['Album_Name','Artist','Album_ID','Album_Name_Spotify','Artists_Spotify'])

In [7]:
#albums_id_3.to_csv("./data/albums_id_3.csv", index=False)

albums_id_3 = pd.read_csv("../data/albums_id_3.csv")
albums_id_3.sample(3)

,Album_Name,Artist,Album_ID,Album_Name_Spotify,Artists_Spotify
67,KinderAngst,KinderAngst,3inNtbWFV5y9Cqr2MZUjxQ,Kinder Angst,['KinderAngst']
6,Swing Around the World,Various Artists,1X70XfhXt2okvamjvuZXVK,Swing Around the World,['Fleur Seule']
11,Dora's World Adventure!,Various Artists,0WDSy20FE73C0f3QR3x73M,Dora The Explorer World Adventure,['Dora The Explorer']


In [13]:
results = []
t1 = time.time()

found = albums_id_1[['Album_Name','Artist']].values.tolist() + albums_id_2[['Album_Name','Artist']].values.tolist()
found += singles_id_1[['Album_Name', 'Artist']].values.tolist()
found += albums_id_3[['Album_Name','Artist']].values.tolist()

print (f"Found {len(found)} albums; {len(album_ratings)-len(found)} albums not found yet.")

Found 1745 albums; 461 albums not found yet.


In [ ]:
for (i, row) in album_ratings[200:].iterrows():
    
    if i%400 ==0 :
        time_elapsed = (time.time()-t1)/60
        print (f"Processing {i}th row.  Time elapsed: {time_elapsed:.2f} minutes. Found {len(results)} singles.")
        
    album_name, artist_name = row.Album_Name, row.Artist
    
    if ([album_name, artist_name] not in found) and ('cd single' not in album_name.lower()):  
        result = search_album(album_name, artist_name, loose_query = True, manual_match = True)
        if result: 
            results.append((album_name, artist_name, *result))
        
    
print (f"Semi-manually Found {len(results)} albums.")

albums_id_4 = pd.DataFrame(results,columns=['Album_Name','Artist','Album_ID','Album_Name_Spotify','Artists_Spotify'])

In [9]:
##albums_id_4.to_csv("./data/albums_id_4.csv", index=False)

albums_id_4 = pd.read_csv("../data/albums_id_4.csv")

albums_id_4.sample(3)

,Album_Name,Artist,Album_ID,Album_Name_Spotify,Artists_Spotify
1,Making Silly Faces,Nick Deysher,2MPgd67X25SmFa01Gj6XRZ,Making Silly Faces,['In The Nick of Time']
0,The Fresh Beat Band: Music from the Hit TV Sho...,The Fresh Beat Band,6jXrznEcmVQ741Kq7zDbQg,The Fresh Beat Band Vol 2.0: More Music From T...,['The Fresh Beat Band']
2,Dev2.0,Dev2.0,5yOLkUelCWvz7pNb3Uyvu6,Devo 2.0,['Devo 2.0']


# Summary: number of albums found

In [32]:
import pandas as pd
albums_id_1 = pd.read_csv("../data/albums_id_1.csv")
albums_id_2 = pd.read_csv("../data/albums_id_2.csv")

albums_id_3 = pd.read_csv("../data/albums_id_3.csv")
albums_id_4 = pd.read_csv("../data/albums_id_4.csv")
singles_id= pd.read_csv("../data/single_ids.csv")

In [33]:
albums_df = pd.concat([albums_id_1, albums_id_2, albums_id_3, albums_id_4])
print (albums_df.shape[0], "albums")
print (singles_id_1.shape[0], "singles")
print ("total: albums + singles = ", albums_df.shape[0] + singles_id.shape[0])

display(albums_df.sample(3))
display(singles_id.sample(3))

1181 albums
567 singles
total: albums + singles =  1748


,Album_Name,Artist,Album_ID,Album_Name_Spotify,Artists_Spotify
447,Right Where You Want Me,Jesse McCartney,2fcFCBEV9lCi8eQeLJIDYA,Right Where You Want Me,['Jesse McCartney']
862,The Music of Grand Theft Auto IV,Various Artists,0lTknHCG5YS0NeVWZuiWzm,The Music of Grand Theft Auto IV,['Various Artists']
649,Raditude,Weezer,6J2BJd3FQDDlh2ZjL6QIpF,Raditude (Deluxe),['Weezer']


,Album_Name,Artist,Album_ID,Album_Name_Spotify,Artists_Spotify,Track_ID,Track_Name_Spotify
168,"""Gonna Get Over You"" (CD Single)",Sara Bareilles,627ukPRwYxyBREHxBq0vGJ,Kaleidoscope Heart,['Sara Bareilles'],45ou2UBThJA4WtFGIiYLI3,Gonna Get Over You
54,"""Hollywood's Not America"" (CD single)",Ferras,2fYGqY3ELfZ5goVJJjXIWi,Aliens & Rainbows,['Ferras'],5wizvr1JlRR3XWmKkOmEtV,Hollywood's Not America - Matrix Version
204,"""State of Grace"" (CD Single)",Taylor Swift,6kZ42qRrzov54LcAk4onW9,Red (Taylor's Version),['Taylor Swift'],6lzc0Al0zfZOIFsFvBS1ki,State Of Grace (Taylor's Version)


In [30]:
albums_df.to_csv("../data/album_ids.csv", index = False)